In [1]:
import numpy as np
import tensorflow as tf

from model import SiameseNet
from data_utils import QuoraDataset, DataIterator
from config import Config
from embeddings import load_embeddings

/home/bruno/Apps/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
### Loading config and pretrained Glove embeddings
config = Config()
loaded_embeddings, (w2idx, idx2w) = load_embeddings(config.glove_filename, binary=False)

Loading from saved word_embeddings
Loading vocab


In [3]:
### Loading Quora Datasets
qd_train = QuoraDataset(config.train_filename, save_path=config.train_save)
w2idx_train, idx2w_train = qd_train.w2idx, qd_train.idx2w

embeddings = np.random.normal(scale=0.001, size=(len(w2idx_train), config.we_dim))

In [4]:
for w, i in w2idx_train.items():
    idx = w2idx.get(w)
    if idx is not None:
        embeddings[i] = loaded_embeddings[idx]

In [5]:
qd_dev  = QuoraDataset(config.dev_filename, w2idx=w2idx_train, save_path=config.dev_save)
qd_test = QuoraDataset(config.test_filename, w2idx=w2idx_train, save_path=config.test_save)

In [6]:
len(qd_train.data()[0])

384289

In [7]:
len(qd_dev.data()[0])

9999

In [8]:
len(qd_test.data()[0])

9999

In [16]:
qids_train = qd_train.q_dict.keys()
qids_dev = qd_dev.q_dict.keys()

In [10]:
len(qids_train)

516663

In [11]:
qids_test = qd_test.q_dict.keys()
len(qids_test)

19372

In [12]:
new_ids_test = [id for id in qids_test if not id in qids_train]

In [13]:
len(new_ids_test)

10733

In [17]:
len(qids_train) + len(qids_test) + len(qids_dev)

555414

In [14]:
test_df = qd_test.df
test_df[:10]

,id,qid1,qid2,question1,question2,is_duplicate
22,22,45,46,What are the questions should not ask on Quora?,Which question should I ask on Quora?,0
28,28,57,58,What is best way to make money online?,What is best way to ask for money online?,0
91,91,183,184,"National Institute of Technology, Kurukshetra:...",National Institute of Technology Karnataka (NI...,0
93,93,187,188,What causes a nightmare?,What causes nightmares that seem real?,1
145,145,291,292,Does Fab currently offer new employees stock o...,Does Uber currently offer new employees stock ...,0
160,160,321,322,What is the funniest joke you know?,What is the funniest joke of all time?,1
164,164,329,330,How can I transfer money from Skrill to a PayP...,How can I send money from my PayPal account to...,0
169,169,339,340,How do you make life suit you and stop life fr...,Why are emotionally abusive people in my life?...,0
198,198,397,398,How can I become more fluent in Chinese?,How can I become fluent in chinese?,1
213,213,427,428,What are the disadvantages of listing a compan...,What are the benefits of listing a company in ...,0


In [15]:
test_df[test_df["qid1"].isin(qids_train)]

,id,qid1,qid2,question1,question2,is_duplicate
22,22,45,46,What are the questions should not ask on Quora?,Which question should I ask on Quora?,0
28,28,57,58,What is best way to make money online?,What is best way to ask for money online?,0
93,93,187,188,What causes a nightmare?,What causes nightmares that seem real?,1
160,160,321,322,What is the funniest joke you know?,What is the funniest joke of all time?,1
164,164,329,330,How can I transfer money from Skrill to a PayP...,How can I send money from my PayPal account to...,0
198,198,397,398,How can I become more fluent in Chinese?,How can I become fluent in chinese?,1
220,220,441,442,How do I earn from Quora?,Can I earn money on Quora?,1
236,236,473,474,How do you potty train a 4 months Pitbull?,How do I potty train my two-month-old Labrador...,1
264,264,81,529,Why do Slavs squat?,Do squats work for men?,0
397,397,792,793,What is a good song to lyric prank your best f...,What are some good lyric prank songs to send y...,1


In [20]:
qd_train.df.is_duplicate.value_counts()

0    242563
1    141726
Name: is_duplicate, dtype: int64